In [1]:
from utilities import *
from env_bruteforce import *
from gis.gis_handler import GISHandler

all_vars = ['x_wec','x_type_wec','x_pen','p_pen','x_env','p_env','p_wec','p_fish_salmon','pos_env', 'gis_handler', 'p_vessel']

conditions = {'current [m/s]': '../data/Surface Current (m-s).tif',
              'oxygen [mg/l]': '../data/Surface Oxygen (mg-l).tif',
              'salinity [PSU]': '../data/Surface Salinity (PSU).tif',
              'temperature [°C]': '../data/Surface Temperature (C).tif',
              'period [s]': '../data/Wave Energy Period (s).tif',
              'height [m]': '../data/Wave Significant Height (m).tif',
              'bathymetry [m]': '../data/Bathymetry (m).tif',
              'distance to port [m]': '../data/Distance to Port (m).tif'}

# high fishing is above average, very high is more than one standard deviation above average
conflicts = {'very high fishing traffic': '../data/Very High Fishing Vessel Traffic.geojson',
#            'high fishing traffic': 'data/High Fishing Vessel Traffic.geojson',
             'marine protected areas': '../data/Marine Protected Areas.geojson',
             'danger zones': '../data/Danger Zones and Restricted Areas.geojson',
             'submarine': '../data/Submarine Transit Lanes.geojson',
             'torpex': '../data/Cape Cod TORPEX.geojson',
             'block island': '../data/Block Island Renewable Energy Zone.geojson',
             'ma wind': '../data/MA Wind Energy Areas.geojson',
             'wind lease': '../data/Potential Wind Lease Areas.geojson',
             'wind planning': '../data/Wind Planning Areas.geojson',
             'shipping': '../data/Shipping Lanes.geojson'}

waters = "../data/Northeast State and Federal Waters.geojson"

handler = GISHandler(conditions, conflicts, waters)

args = {}
args['handler'] = handler

args['grid_resolution'] = 0.1
best_x_total, aqua_obj_best_total, p_best, best_x_without_conflict, aqua_obj_best_without_conflict = env_bruteforce(all_vars, args)
bruteforce_result("without conflict consideration", best_x_total, aqua_obj_best_total, p_best)
bruteforce_result("with conflict consideration", best_x_without_conflict, aqua_obj_best_without_conflict, p_best)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
brute force without conflict consideration
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
design variable:
   pos_lat       43.950 m
   pos_long     -68.480 m
----------------------------------------
 objective function terms:
   Objective_func 109680868.339
   fish_yield     2162986.542 [kg]
----------------------------------------
 wave energy converter:
   wec number         18.000 [-]
   wave power          3.373 [kW/m]
   wec_P_ave           7.841 [kW]
   wec AEP        1237228.302 [kWh]
----------------------------------------
 price break down:
   wec NPV              109131900.357 [$]
   pen price            12723450.247 [$]
   fish feed price      5649211.910 [$]
   vessel travel NPV    548967.982 [$]
----------------------------------------
 location validation:
   conditions           True
   scope                True
   conflicts            False
++++++++++++++